
# LeNet‑5 sur AMHCD (RGB) — Notebook

**Objectif** : entraîner et évaluer une implémentation Keras de LeNet‑5 adaptée à **32×32×3** sur le dataset **AMHCD** (Tifinagh manuscrit).

> **Remarque importante** : ce notebook n'a pas d'accès internet ici. Téléchargez d'abord le dataset depuis Kaggle
> (https://www.kaggle.com/datasets/benaddym/amazigh-handwritten-character-database-amhcd) puis placez-le localement.
> Deux façons de procéder :
> 1. **Dossiers par classe** : `data/AMHCD/train/<classe>/*.png`, `data/AMHCD/val/<classe>/*.png`, `data/AMHCD/test/<classe>/*.png`
> 2. **Un seul dossier** : `data/AMHCD/all/<classe>/*.png` (on fera le split stratifié dans le notebook).


In [ ]:

import os, random, math, json, itertools, time, shutil, glob
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

# Reproductibilité
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(tf.__version__)
device_name = tf.test.gpu_device_name()
print("GPU:", device_name if device_name else "CPU only")


In [ ]:

# === CONFIG UTILISATEUR ===
CFG = {
    "root_dir": "data/AMHCD",   # dossier racine
    "train_subdir": "train",    # si vous avez déjà un split
    "val_subdir": "val",
    "test_subdir": "test",
    "all_subdir": "all",        # sinon, mettez tout ici par classe
    "img_size": 32,
    "batch_size": 64,
    "epochs": 30,
    "augment": True,
    "optimizer": "adam",        # "sgd" ou "adam"
    "initial_lr": 1e-3,         # 1e-2 si SGD
    "use_maxpool": True,        # False -> AveragePooling
    "use_relu": True,           # False -> tanh
    "dropout": 0.5,
    "weight_decay": 0.0,        # L2 si > 0
    "patience": 7
}
CFG


In [ ]:

from typing import Tuple, List, Dict
import pathlib
import PIL
from PIL import Image

AUTOTUNE = tf.data.AUTOTUNE

def has_pre_split(root):
    return (pathlib.Path(root)/CFG["train_subdir"]).exists() and (pathlib.Path(root)/CFG["val_subdir"]).exists()

def get_class_names(root):
    # Peek into one split (train or all) to infer classes
    base = pathlib.Path(root)/ (CFG["train_subdir"] if has_pre_split(root) else CFG["all_subdir"])
    classes = sorted([p.name for p in base.iterdir() if p.is_dir()])
    return classes

def make_datasets(root:str):
    root = pathlib.Path(root)
    img_size = (CFG["img_size"], CFG["img_size"])
    batch_size = CFG["batch_size"]
    if has_pre_split(root):
        train_dir = root/CFG["train_subdir"]
        val_dir   = root/CFG["val_subdir"]
        test_dir  = root/CFG["test_subdir"]
        train_ds = keras.preprocessing.image_dataset_from_directory(
            train_dir, image_size=img_size, batch_size=batch_size, seed=SEED, label_mode="categorical")
        val_ds = keras.preprocessing.image_dataset_from_directory(
            val_dir, image_size=img_size, batch_size=batch_size, seed=SEED, label_mode="categorical")
        test_ds = keras.preprocessing.image_dataset_from_directory(
            test_dir, image_size=img_size, batch_size=batch_size, seed=SEED, label_mode="categorical", shuffle=False)
        class_names = train_ds.class_names
    else:
        all_dir = root/CFG["all_subdir"]
        all_ds = keras.preprocessing.image_dataset_from_directory(
            all_dir, image_size=img_size, batch_size=batch_size, seed=SEED, label_mode="categorical")
        class_names = all_ds.class_names
        # split 70/15/15
        card = tf.data.experimental.cardinality(all_ds).numpy()
        n_train = int(0.7 * card)
        n_val   = int(0.15 * card)
        train_ds = all_ds.take(n_train)
        rem = all_ds.skip(n_train)
        val_ds = rem.take(n_val)
        test_ds = rem.skip(n_val)
    num_classes = len(class_names)
    print("Classes:", class_names)
    print("N classes:", num_classes)

    def norm(x,y): return (tf.cast(x, tf.float32)/255.0, y)
    train_ds = train_ds.map(norm, num_parallel_calls=AUTOTUNE)
    val_ds   = val_ds.map(norm,   num_parallel_calls=AUTOTUNE)
    test_ds  = test_ds.map(norm,  num_parallel_calls=AUTOTUNE)

    if CFG["augment"]:
        aug = keras.Sequential([
            layers.RandomRotation(0.05, fill_mode="nearest"),
            layers.RandomTranslation(0.05,0.05, fill_mode="nearest"),
            layers.RandomZoom(0.05),
            layers.RandomBrightness(factor=0.1)
        ], name="augmentation")
        train_ds = train_ds.map(lambda x,y: (aug(x, training=True), y), num_parallel_calls=AUTOTUNE)

    train_ds = train_ds.shuffle(1000, seed=SEED).prefetch(AUTOTUNE)
    val_ds   = val_ds.prefetch(AUTOTUNE)
    test_ds  = test_ds.prefetch(AUTOTUNE)
    return train_ds, val_ds, test_ds, class_names


In [ ]:

def build_lenet5(input_shape=(32,32,3), num_classes=33):
    Pool = layers.MaxPooling2D if CFG["use_maxpool"] else layers.AveragePooling2D
    Act  = layers.ReLU if CFG["use_relu"] else layers.Activation
    wd = CFG["weight_decay"]
    reg = regularizers.l2(wd) if wd and wd>0 else None

    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(6, kernel_size=5, padding="valid", kernel_regularizer=reg)(inputs)
    x = (Act() if CFG["use_relu"] else Act("tanh"))(x)
    x = Pool(pool_size=2, strides=2)(x)

    x = layers.Conv2D(16, kernel_size=5, padding="valid", kernel_regularizer=reg)(x)
    x = (Act() if CFG["use_relu"] else Act("tanh"))(x)
    x = Pool(pool_size=2, strides=2)(x)

    x = layers.Conv2D(120, kernel_size=5, padding="valid", kernel_regularizer=reg)(x)
    x = (Act() if CFG["use_relu"] else Act("tanh"))(x)

    x = layers.Flatten()(x)
    x = layers.Dense(84, kernel_regularizer=reg)(x)
    x = (Act() if CFG["use_relu"] else Act("tanh"))(x)
    if CFG["dropout"] and CFG["dropout"]>0:
        x = layers.Dropout(CFG["dropout"])(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs, name="LeNet5_AMHCD")


In [ ]:

root = CFG["root_dir"]
train_ds, val_ds, test_ds, class_names = make_datasets(root)
num_classes = len(class_names)

model = build_lenet5(input_shape=(CFG["img_size"], CFG["img_size"], 3), num_classes=num_classes)
model.summary()

if CFG["optimizer"].lower() == "sgd":
    opt = keras.optimizers.SGD(learning_rate=CFG["initial_lr"], momentum=0.9, nesterov=True)
else:
    opt = keras.optimizers.Adam(learning_rate=CFG["initial_lr"])

model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy", keras.metrics.TopKCategoricalAccuracy(k=3, name="top3")])

ckpt_path = "lenet5_amhcd_best.keras"
callbacks = [
    keras.callbacks.ModelCheckpoint(ckpt_path, monitor="val_accuracy", save_best_only=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=max(2, CFG["patience"]//2), verbose=1),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=CFG["patience"], restore_best_weights=True, verbose=1)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=CFG["epochs"], callbacks=callbacks)
model.save("lenet5_amhcd_final.keras")


In [ ]:

import matplotlib.pyplot as plt

h = history.history
# Accuracy
plt.figure()
plt.plot(h["accuracy"], label="train_acc")
plt.plot(h["val_accuracy"], label="val_acc")
plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend(); plt.title("Accuracy")
plt.savefig("curve_accuracy.png", dpi=150)
plt.show()

# Loss
plt.figure()
plt.plot(h["loss"], label="train_loss")
plt.plot(h["val_loss"], label="val_loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.title("Loss")
plt.savefig("curve_loss.png", dpi=150)
plt.show()

# Top-3
plt.figure()
plt.plot(h["top3"], label="train_top3")
plt.plot(h["val_top3"], label="val_top3")
plt.xlabel("Epoch"); plt.ylabel("Top-3 Acc"); plt.legend(); plt.title("Top-3 Accuracy")
plt.savefig("curve_top3.png", dpi=150)
plt.show()


In [ ]:

from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Évaluation
test_loss, test_acc, test_top3 = model.evaluate(test_ds, verbose=0)
print(f"Test accuracy: {test_acc:.4f} | Top-3: {test_top3:.4f} | Loss: {test_loss:.4f}")

# Prédictions
y_true = []
for _, y in test_ds:
    y_true.append(np.argmax(y.numpy(), axis=1))
y_true = np.concatenate(y_true, axis=0)

y_pred_prob = model.predict(test_ds, verbose=0)
y_pred = np.argmax(y_pred_prob, axis=1)

print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

cm = confusion_matrix(y_true, y_pred)
np.save("confusion_matrix.npy", cm)

# Affichage matrice de confusion (matplotlib, sans style/couleurs personnalisés)
plt.figure(figsize=(8,8))
plt.imshow(cm, interpolation="nearest")
plt.title("Matrice de confusion")
plt.colorbar()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=90)
plt.yticks(tick_marks, class_names)
plt.tight_layout()
plt.xlabel("Prédit")
plt.ylabel("Vrai")
plt.savefig("confusion_matrix.png", dpi=200, bbox_inches="tight")
plt.show()


In [ ]:

# Grille d'exemples correctement et mal classés
import math

# Récupérer les images et labels originaux du test_ds
imgs = []
labels = []
for x, y in test_ds.unbatch().take(200):  # limite pour performance
    imgs.append(x.numpy())
    labels.append(np.argmax(y.numpy()))
imgs = np.stack(imgs, axis=0)
labels = np.array(labels)

preds = np.argmax(model.predict(imgs, verbose=0), axis=1)

correct_idx = np.where(preds == labels)[0][:25]
wrong_idx   = np.where(preds != labels)[0][:25]

def plot_grid(indices, title, filename):
    n = len(indices)
    cols = 5
    rows = int(math.ceil(n/cols))
    plt.figure(figsize=(cols*2, rows*2))
    for i, idx in enumerate(indices):
        plt.subplot(rows, cols, i+1)
        plt.imshow(imgs[idx])
        plt.axis("off")
        plt.title(f"GT:{class_names[labels[idx]]}\nP:{class_names[preds[idx]]}")
    plt.suptitle(title)
    plt.savefig(filename, dpi=150, bbox_inches="tight")
    plt.show()

plot_grid(correct_idx, "Exemples bien classés", "qualitative_correct.png")
plot_grid(wrong_idx,   "Exemples mal classés", "qualitative_wrong.png")


In [ ]:

summary = {
    "test_accuracy": float(test_acc),
    "test_top3": float(test_top3),
    "classes": class_names,
    "config": CFG
}
with open("results_summary.json","w") as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print("Artifacts enregistrés:",
      ["lenet5_amhcd_best.keras", "lenet5_amhcd_final.keras",
       "curve_accuracy.png","curve_loss.png","curve_top3.png",
       "confusion_matrix.png","qualitative_correct.png","qualitative_wrong.png",
       "results_summary.json"])


In [ ]:

# (Optionnel) Grad-CAM simple sur la dernière couche conv (Conv2D_120)
# Pour aller vite, applique sur une seule image.
def grad_cam(model, img_array, last_conv_layer_name=None, class_index=None):
    if last_conv_layer_name is None:
        # heuristique: trouver la dernière Conv2D
        last_conv_layer_name = None
        for layer in reversed(model.layers):
            if isinstance(layer, layers.Conv2D):
                last_conv_layer_name = layer.name
                break
    conv_layer = model.get_layer(last_conv_layer_name)

    grad_model = keras.models.Model([model.inputs], [conv_layer.output, model.output])
    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(img_array, training=False)
        if class_index is None:
            class_index = tf.argmax(preds[0])
        class_channel = preds[:, class_index]

    grads = tape.gradient(class_channel, conv_out)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_out = conv_out[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_out), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

# Démo (si test_ds non vide)
for x, y in test_ds.take(1):
    img = x[0:1]
    hm = grad_cam(model, img)
    import matplotlib.pyplot as plt
    plt.figure()
    plt.imshow(x[0].numpy())
    plt.imshow(hm, alpha=0.3)
    plt.title("Grad-CAM (superposé)")
    plt.savefig("gradcam_example.png", dpi=150, bbox_inches="tight")
    plt.show()
    break
